In [11]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

%load_ext autoreload
%autoreload all
    
from utils import read_images
from datasets import DominanceDataset
from models import SiameseResNet

torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model = SiameseResNet()
model.load_state_dict(torch.load('weights/dominance.pth'))
model.eval()

images = read_images(ids="test")
test_dataset = DominanceDataset(images, split="test")
#test_loader = DataLoader(test_dataset, batch_size=8)

Loading imgs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 367/367 [00:46<00:00,  7.87it/s]


NameError: name 'DataLoader' is not defined

In [9]:
def get_pred(left_imgs, right_imgs):
    preds = []
    for l in left_imgs:
        for r in right_imgs:
            pred = torch.sigmoid(model(l, r))[0, 1]
            preds.append(pred)
    preds = np.array(preds)
    return preds.mean(), preds.std()

In [13]:
# eval representative frame sampling 

labels = []
preds = []
for i in range(len(test_dataset)):
    study_id, data = test_dataset.index[i]

    imgs_left=[]
    for img_id, frame_n in zip(data["l"], data["l_f"]):
        img = images[img_id][frame_n]
        img = test_dataset.transform(img).unsqueeze(0)
        imgs_left.append(img)

    imgs_right=[]
    for img_id, frame_n in zip(data["r"], data["r_f"]):
        img = images[img_id][frame_n]
        img = test_dataset.transform(img).unsqueeze(0)
        imgs_right.append(img)

    pred, _ = get_pred(imgs_left, imgs_right)
    preds.append(1 if pred > 0.5 else 0)
    labels.append(data["label"])

report = classification_report(labels, preds)
matrix = confusion_matrix(labels, preds)
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(matrix)

Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        16

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16

Confusion Matrix:
[[16]]


/root/przemek/miniconda3/envs/torch21/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
